<a href="https://colab.research.google.com/github/bibidemarezoyens/LB/blob/main/rowrowrow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
complete_df = pd.read_excel('data_roeien.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'data_roeien.xlsx'

In [ ]:
def opschonen(data):
    ### Onnodige kolommen verwijderen
    complete_df = pd.read_excel(data)
    clean_col_df = complete_df.drop(columns=['datum', 'ploeg', 'zone', 'intervaltype', 'interval_afstand', 'interval_tijd', 'interval_nummer', 'rust', 'machine', 'spm', '2k datum'], errors='ignore')

    ### Lege strings omzetten naar NaN in de specifieke kolommen
    clean_col_df['500_split'] = clean_col_df['500_split'].replace('', pd.NA)
    clean_col_df['2k tijd'] = clean_col_df['2k tijd'].replace('', pd.NA)

    ### Alleen rijen met lege '500_split' of '2k tijd' verwijderen
    clean_row_df = clean_col_df.dropna(subset=['500_split', '2k tijd'])
    clean_row_df.dropna(how='any',inplace=True)

    ### Maak een kopie voor opschonen
    clean_df = clean_row_df.copy()
    clean_df.dropna(subset=['500_split', '2k tijd'])

    ### Komma's naar punten veranderen in ‘500_split’ en ‘2k tijd’ columns en uren veranderen naar minuten (00:01:11.11 wordt 1:11.11)
    for col in ['500_split', '2k tijd']:
        print(clean_df[col])
        clean_df[col] = clean_df[col].astype(str).str.strip()
        clean_df[col] = clean_df[col].str.replace(',', '.', regex=False)
        clean_df[col] = clean_df[col].str.lstrip('0')

        # clean_df[col] = clean_df[col].astype(str).str.strip()
        # clean_df[col] = clean_df[col].str.replace(',', '.', regex=False)
        # clean_df[col] = clean_df[col].str.lstrip('0')
        # clean_df[col] = clean_df[col].str.replace('^0:0', '', regex=True)
        print(clean_df[col])

    ### Verander 'trainingype' naar 'trainingtype'
    clean_df = clean_df.rename(columns={'trainingype': 'trainingtype'})

    return clean_df

In [ ]:
### de minuten veranderen naar seconden

def time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    try:
        minutes, seconds = map(float, str(time_str).split(':'))
        print('GOOD:', time_str)
        return minutes * 60 + seconds
    except ValueError:
        print('ERROR:', time_str)
        return None

def verander_omzetting_seconden(docu):
    docu['500_split'] = docu['500_split'].apply(time_to_seconds)
    docu['2k tijd'] = docu['2k tijd'].apply(time_to_seconds)
    return docu


In [ ]:
groot_roeien_opgeschoond = opschonen('data_roeien.xlsx')
klein_roeien_opgeschoond = opschonen('klein_roeien.xlsx')
# print(groot_roeien_opgeschoond['500_split'].isnull().sum())


In [ ]:
print(groot_roeien_opgeschoond['500_split'])

In [ ]:
groot_dataframe_op_sec = verander_omzetting_seconden(groot_roeien_opgeschoond)
klein_dataframe_op_sec = verander_omzetting_seconden(klein_roeien_opgeschoond)

print(groot_dataframe_op_sec['500_split'].isnull().sum())
print(groot_roeien_opgeschoond['500_split'].isnull().sum())

In [ ]:
print(groot_dataframe_op_sec['500_split'].isnull().sum())
print(groot_dataframe_op_sec[groot_dataframe_op_sec['500_split'].isnull()])
groot_dataframe_op_sec.dropna(inplace=True)
print(groot_dataframe_op_sec['500_split'].isnull().sum())
print(groot_dataframe_op_sec['2k tijd'].isnull().sum())
print(groot_dataframe_op_sec)

groot_dataframe_op_sec.to_csv('verschoond_dataframe.csv', index=False)

In [ ]:
def omzetten_trainingstype(data):
    df = pd.read_excel(data)
    print(df['trainingtype'].unique())

    df['trainingtype'] = df['trainingtype'].str.replace(r"/[a-zA-Z0-9']{3}", "", regex=True)
    df['trainingtype'] = df['trainingtype'].str.replace("1'", 'minuutjes', regex=False)
    df['trainingtype'] = df['trainingtype'].str.replace("1minuutjes", "11'", regex=False)
    df['trainingtype'] = df['trainingtype'].str.replace("'", "", regex=False)
    df['trainingtype'] = df['trainingtype'].str.replace(" ", "", regex=False)

    df['trainingtype'] = df['trainingtype'].str.replace("r", "", regex=False)



    # df['trainingtype'] = df['trainingtype'].str.replace("3x15'", '2', regex=False)
    # df['trainingtype'] = df['trainingtype'].str.replace("3x2000m/5'r", '3', regex=False)
    # df['trainingtype'] = df['trainingtype'].str.replace("5x5'", '1', regex=False)
    # df['trainingtype'] = df['trainingtype'].str.replace("5x5'", '1', regex=False)
    print(df['trainingtype'].unique())

    # df['trainingtype'] = pd.to_numeric(df['trainingtype'], errors='coerce')
    # print(df['trainingtype'].unique())  # Inspect unique values
    # df['trainingtype'].str.replace("5x5'", "1" , regex=False)
    return df

In [ ]:
print(omzetten_trainingstype('verschoond_dataframe.xlsx'))

In [ ]:
### per persoon per training de gemiddelde 500_split berekenen

def gemid_split(data):

    gemid_data = (data.groupby(['naam', 'trainingtype'], as_index=False).agg({'ervaring': 'first', 'geslacht': 'first', 'gewichtsklasse': 'first', '500_split': 'mean', '2k tijd': 'first'}))

    return gemid_data

klein_gemid_df = gemid_split(klein_dataframe_op_sec)
print(klein_gemid_df)
groot_gemid_df = gemid_split(groot_dataframe_op_sec)
print(groot_gemid_df)

In [ ]:
### een dictionary maken van alle gegevens per persoon

import json

def dict_per_naam(data):
  naam_dict = {}

  for i in range(len(data)):
      row = data.iloc[i].to_dict()

      name = row.pop('naam')
      constants = {key: row.pop(key) for key in ['ervaring', 'geslacht', 'gewichtsklasse', '2k tijd']}

      if name not in naam_dict:
          naam_dict[name] = {'gegevens': constants, 'tijden': []}

      naam_dict[name]['tijden'].append(row)

  print(json.dumps(naam_dict, indent=4, sort_keys=False))

  return naam_dict

klein_namen_dict = dict_per_naam(klein_dataframe_op_sec)
gemid_klein_namen_dict = dict_per_naam(klein_gemid_df)

In [ ]:
# groot_namen_dict = dict_per_naam(groot_dataframe_op_sec[:1000])
# gemid_groot_namen_dict = dict_per_naam(groot_gemid_df)

gemid_klein_namen_dict = dict_per_naam(klein_gemid_df)

In [ ]:
from sklearn.model_selection import train_test_split

# # Data inladen
# df = groot_gemid_df

# # Eerst de data opschudden om bias te voorkomen
# df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# # Zorg ervoor dat elke ervaring, geslacht en gewichtsklasse in elke set vertegenwoordigd zijn
# train_data = pd.DataFrame()
# val_data = pd.DataFrame()
# test_data = pd.DataFrame()

# for ervaring in df['ervaring'].unique():
#     for geslacht in df['geslacht'].unique():
#         for gewichtsklasse in df['gewichtsklasse'].unique():
#             subset = df[(df['ervaring'] == ervaring) & (df['geslacht'] == geslacht) & (df['gewichtsklasse'] == gewichtsklasse)]
#             if not subset.empty:
#                 temp_train, temp_temp = train_test_split(subset, test_size=0.3, random_state=42)
#                 temp_val, temp_test = train_test_split(temp_temp, test_size=0.5, random_state=42)
#                 train_data = pd.concat([train_data, temp_train])
#                 val_data = pd.concat([val_data, temp_val])
#                 test_data = pd.concat([test_data, temp_test])

# # Reset indexen
# train_data.reset_index(drop=True, inplace=True)
# val_data.reset_index(drop=True, inplace=True)
# test_data.reset_index(drop=True, inplace=True)

# # Controleren op juiste verdeling
# print(f"Trainingsdata: {len(train_data)} rijen")
# print(f"Validatiedata: {len(val_data)} rijen")
# print(f"Testdata: {len(test_data)} rijen")

# # Optioneel: data opslaan in aparte bestanden
# train_data.to_csv('train_data.csv', index=False)
# val_data.to_csv('val_data.csv', index=False)
# test_data.to_csv('test_data.csv', index=False)

In [ ]:
def tweek_naar_split(groot_dataframe_op_sec):
    for i in range(len(groot_dataframe_op_sec['500_split'])):
        if groot_dataframe_op_sec['500_split'].iloc[i] > 300:
          groot_dataframe_op_sec['500_split'].iloc[i] = groot_dataframe_op_sec['500_split'].iloc[i] / 4
    return groot_dataframe_op_sec['500_split']

In [ ]:
alles = tweek_naar_split(groot_dataframe_op_sec)
print(alles)
print(len(alles))

min_value = alles.min()
filtered_alles = alles[alles!= min_value]

unique_values = alles.unique()
unique_values.sort()

first_min = unique_values[0]
second_min = unique_values[1]
third_min = unique_values[2]
fourth_min = unique_values[3]
eighth_min = unique_values[7]

print(first_min)
print(second_min)
print(third_min)
print(fourth_min)
print(eighth_min)

unique_values_desc = alles.unique()
unique_values_desc.sort()
unique_values_desc = unique_values_desc[::-1]

first_largest = unique_values_desc[0]
second_largest = unique_values_desc[1]
third_largest = unique_values_desc[2]
eighth_largest = unique_values_desc[7]

print(first_largest)
print(second_largest)
print(third_largest)
print(eighth_largest)

In [ ]:
def remove_outliers(groot_dataframe_op_sec):
  alles = tweek_naar_split(groot_dataframe_op_sec)
  alles = alles[alles >= 80]
  return alles
print(len(alles))